# Классификация фитнесов по площади

Код лучше запускать в виртуальной среде на мощной GPU `(kaggle/collab)` (иначе будет долго грузиться),

P.S. Мне больше импонирует `kaggle`, в `collab` все часто вылетает, но в данном кейсе не требуется много памяти, поэтому платформа не так важна.

In [1]:
from accelerate.test_utils.testing import get_backend
device, _, _ = get_backend()
device

'cuda'

In [2]:
!pip install -U transformers

In [3]:
# Если работаете в kaggle, то надо запустить эту ячейку, чтобы подключиться к своему гугл диску и импортировать оттуда файлы
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import time
import json
import requests
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.metrics import classification_report, precision_score
from sklearn.metrics import recall_score, f1_score, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import MinMaxScaler
from PIL import Image, UnidentifiedImageError
from transformers import CLIPProcessor, CLIPModel, pipeline
import torch
import torchvision.transforms as transforms
import nltk
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
from tqdm import tqdm

### Загружаем файл

In [13]:
city = pd.read_csv('/content/drive/MyDrive/FitnessData/Кейсы/Фото_фитнес_азия/Джакарта/jakarta_fitness_itog.csv')

### Проверяем, есть ли объекты за пределами города

In [ ]:
pip install osmnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 3.4 MB/s eta 0:00:00


In [14]:
len(city)

2154

In [ ]:
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point

city_boundary = "Chennai, India"
boundary = ox.geocode_to_gdf(city_boundary)
boundary = boundary.to_crs(3857)

def coordinates_to_geometry(row):
    return Point(row['longitude'], row['latitude'])

city['geometry'] = city.apply(coordinates_to_geometry, axis=1)
city_gdf = gpd.GeoDataFrame(city, geometry='geometry', crs='EPSG:4326')

print(f'Length before clipping {len(city_gdf)}')
if city_gdf.crs != boundary.crs:
    city_gdf  = city_gdf.to_crs(boundary.crs)
city_gdf_clipped = gpd.clip(city_gdf, boundary)
print(f'Length after clipping {len(city_gdf_clipped)}')

city = city_gdf_clipped.drop(columns='geometry')

### Если есть дубликаты в данных
- Удаляем сначала повторяющиеся строчки
- Ищем одинаковые адреса, объединяем уникальные фото, привязанные к ним
- Оставляем наибольшее значение количества отзывов `review_count` для каждого адреса

In [15]:
def extract_images_and_titles(image_link_str):
    try:
        image_data = json.loads(image_link_str)
        return [item['image'] for item in image_data]
    except json.JSONDecodeError:
        return []

def to_correct_links(links):
    global links_lst
    lst = []
    for link in links:
        if link.startswith('https:'):
            link = link
        else:
            link = 'https:'+link
        links_lst.append(link)
        lst.append(link)
    return lst

links_lst = []

city = city.drop_duplicates().reset_index(drop=True)
city['images_links'] = city['images'].apply(extract_images_and_titles)
city['images_links'] = city['images_links'].apply(to_correct_links)

In [16]:
def extract_place_id(url: str):
    if not isinstance(url, str):
        return None
    m = re.search(r'(?:placeid|place_id)=([^&]+)', url)
    return m.group(1) if m else None

city['placeid'] = city['reviews_link'].apply(extract_place_id)
city['placeid'] = city['placeid'].fillna('no')

In [17]:
dictt = {}
for index, row in city.iterrows():
    unique_obj = row['data_id']
    if len(row['images_links']) > 0:
        if unique_obj in dictt:
            dictt[unique_obj] |= set(row['images_links'])
        else:
            dictt[unique_obj] = set(row['images_links'])
    else:
        if unique_obj not in dictt:
            dictt[unique_obj] = set()

def to_all_imgs(ad):
    return list(dictt[ad])

def to_one_review_count(ad):
    return max(city[city['data_id']==ad]['review_count'])

city['images_links'] = city['data_id'].apply(to_all_imgs)
city['review_count'] = city['data_id'].apply(to_one_review_count)

city = city.drop_duplicates(subset = 'data_id').reset_index(drop = True)

# Можно посмотреть среднее и максимальное количество фотографий, посмотреть на несколько из них
# lst = []
# for i in city['images_links']:
#     lst.append(len(i))

# print(np.mean(lst))
# print(max(lst))

# for index, row in city.sample(5).iterrows():
#     if row['images_links']:  # Проверяем, есть ли изображения
#         for img_url in row['images_links']:
#             display(Image(url=img_url))
#             break

# Лучше сохраниться
# city.to_csv('city.csv', index = False) # Ссылка на файл

### Обрабатываем уже существующие признаки `category`, `about`

In [18]:
city['category_complex'] = city['category'].apply(lambda x: 1 if re.search(r'\b(complex|athletic|stadium)\b', str(x).lower()) else 0)
city['category_yoga'] = city['category'].apply(lambda x: 1 if re.search(r'\b(yoga|gymnastic|pilates|health|dance|shaping|dietitian|spa|diabet|sauna|weight|nutritionist)\b', str(x).lower()) else 0)

In [19]:
def to_studio(value):
    if 'studio' in str(value).lower():
        return 1
    else:
        return 0

def to_pool(value):
    if value is not None:
        value = str(value)
        if '{"name":"Swimming pool","enabled":true}' in value:
            return 1
    return 0

def to_sauna(value):
    if value is not None:
        value = str(value)
        if '{"name":"Sauna","enabled":true}' in value:
            return 1
    return 0

def to_free_parking_lot(value):
    if value is not None:
        value = str(value)
        if '{"name":"Free parking lot","enabled":true}' in value:
            return 1
    return 0

def to_paid_parking_lot(value):
    if value is not None:
        value = str(value)
        if '{"name":"Paid parking lot","enabled":true}' in value:
            return 1
    return 0

def to_free_multi_storey_cp(value):
    if value is not None:
        value = str(value)
        if '{"name":"Free multi-storey car park","enabled":true}' in value:
            return 1
    return 0

def to_paid_multi_storey_cp(value):
    if value is not None:
        value = str(value)
        if '{"name":"Paid multi-storey car park","enabled":true}' in value:
            return 1
    return 0

def to_paid_street_parking(value):
    if value is not None:
        value = str(value)
        if '{"name":"Paid street parking","enabled":true}' in value:
            return 1
    return 0

def to_free_street_parking(value):
    if value is not None:
        value = str(value)
        if '{"name":"Free of charge street parking","enabled":true}' in value:
            return 1
    return 0

city['group_studio'] = city['category'].apply(to_studio)
city['pool'] = city['about'].apply(to_pool)
city['sauna'] = city['about'].apply(to_sauna)

# Признаки, связанные с парковкой я не использовала в итоговой модели, оставляю на случай,
# если они нужны для других задач
city['free_parking_lot'] = city['about'].apply(to_free_parking_lot)
city['paid_parking_lot'] = city['about'].apply(to_paid_parking_lot)
city['free_multi_storey_cp'] = city['about'].apply(to_free_multi_storey_cp)
city['paid_multi_storey_cp'] = city['about'].apply(to_paid_multi_storey_cp)
city['free_street_parking'] = city['about'].apply(to_free_street_parking)
city['paid_street_parking'] = city['about'].apply(to_paid_street_parking)
city['free_parking'] = city[['free_parking_lot',
                                        'free_multi_storey_cp',
                                        'free_street_parking']].apply(lambda x: 1 if x.sum() >= 1 else 0, axis=1)
city['paid_parking'] = city[['paid_parking_lot',
                                             'paid_multi_storey_cp',
                                             'paid_street_parking']].apply(lambda x: 1 if x.sum() >= 1 else 0, axis=1)
city['parking'] = city[['free_parking_lot',
                                        'paid_parking_lot',
                                        'free_multi_storey_cp',
                                        'paid_multi_storey_cp',
                                        'free_street_parking',
                                        'paid_street_parking']].apply(lambda x: 1 if x.sum() >= 1 else 0, axis=1)

#### Теперь отфильтруем только те изображения, где уверенность в информативном классе более или равна 90 % с помощью `Zero-Shot Image Classification` модели [openai/clip-vit-large-patch14](https://huggingface.co/openai/clip-vit-large-patch14)

Важно, чтобы в столбце useful_images был список, а не строка, поэтому если заново импортируете файл (например, при открытии в виртуальной среде), то надо применить функцию convert_to_list ниже

In [ ]:
### Если заново импортируете файл, запустите эту ячейку

def to_correct_links(links):
    global links_lst
    lst = []
    for link in links:
        if link.startswith('https:'):
            link = link
        else:
            link = 'https:'+ link
        links_lst.append(link)
        lst.append(link)
    return lst

def convert_to_list(image_str):
    if image_str == '[]':
        return []
    else:
        return image_str[2:-2].split("', '")

city = pd.read_csv('city.csv') # Ссылка на файл

city['images_links'] = city['images_links'].apply(convert_to_list)
links_lst = []
city['images_links'].apply(to_correct_links)
print('Done')

FileNotFoundError: [Errno 2] No such file or directory: 'city.csv'

In [20]:
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)   # модель → GPU
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

lst_probs = []
lst_useful_links = []

# ──▼ добавили tqdm вокруг links_lst ▼──
for url in tqdm(links_lst, desc="Processing images"):
    try:
        image = Image.open(requests.get(url, stream=True).raw)

        inputs = processor(
            text=[
                "indoor gym fitness center yoga studio pilates pool exercise room sports club with exercise equipment yoga mats mirror",
                "building outdoors or branding sign or trees or other",
            ],
            images=image,
            return_tensors="pt",
            padding=True,
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)

        logits_per_image = outputs.logits_per_image
        prob = round(float(logits_per_image.softmax(dim=1)[0][0]), 2)

        lst_probs.append(prob)
        if prob >= 0.9:
            lst_useful_links.append(url)

    except UnidentifiedImageError as e:
        tqdm.write(f"Could not identify image from {url}: {e}")
        continue
    except Exception as e:
        tqdm.write(f"Error processing {url}: {e}")
        continue

def to_filter_links(links):
    return [link for link in links if link in lst_useful_links]

city["useful_images"] = city["images_links"].apply(to_filter_links)

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

...


Processing images:  86%|████████▌ | 6937/8090 [35:16<02:19,  8.24it/s]

Processing images:  86%|████████▌ | 6939/8090 [35:16<01:58,  9.68it/s]

Could not identify image from https://lh3.googleusercontent.com/gps-proxy/ALd4DhHHHGFMT0E4uUcC68gIIZjljrsntJt3FXd5pcaSR3Yx1LBUgW7q3xMnvUHK0ocvJLvZBomoQ5abKZvr39A3IAgPJJiOEZkmwQ4PtXbvURRqqaqXstPpdce7F9uWvdX0hdh-9a4hcx8a36UiNJ82hcv_rV5kUF6x1-G7xwgFmpOihuS5MIs2WAkkT519lxKKTCpd9w=w397-h298-k-no: cannot identify image file <_io.BytesIO object at 0x7b6f6589fa10>



Processing images:  89%|████████▉ | 7215/8090 [35:58<02:37,  5.56it/s]

Processing images:  89%|████████▉ | 7215/8090 [35:58<02:37,  5.56it/s]

Could not identify image from https://lh3.googleusercontent.com/gps-proxy/ALd4DhH66ZyRDRBl3JET-qcZna9-bNulNeA85WWKpt90i4DySXRGv5xjRz65myH-fyNUXjMvJuwc-xh-B-Cpo8J0MHEwKtpvJkLCwry5O9gE7s2SHX0424dKT_iAiWqQyZW1IFe9Kvo6owwmJ8nYvegm7HHOs_R7oi1CH77Ls8iwFfXOiD_9I1UMQVadDqRyclHRAqowh34=w397-h298-k-no: cannot identify image file <_io.BytesIO object at 0x7b6f657374c0>



Processing images: 100%|██████████| 8090/8090 [38:12<00:00,  3.53it/s]


In [21]:
# Сохранение общего файла
city.to_csv('/content/drive/MyDrive/FitnessData/Кейсы/Фото_фитнес_азия/Джакарта/jakarta_filter_photo.csv', index = False) # Ссылка на файл

# Сохранение списка с "хорошими" ссылками
with open('/content/drive/MyDrive/FitnessData/Кейсы/Фото_фитнес_азия/Джакарта/jakarta_useful_img.txt', 'w') as file: # Ссылка на файл
    for item in lst_useful_links:
        file.write(f"{item}\n")

#### Поработаем с размером комнаты на фотографиях. Отфильтруем только те изображения, где уверенность в большом размере комнаты более или равна 80 % с помощью `Zero-Shot Image Classification` модели [openai/clip-vit-large-patch14](https://huggingface.co/openai/clip-vit-large-patch14)

Важно, чтобы в столбце useful_images был список, а не строка, поэтому если заново импортируете файл (например, при открытии в виртуальной среде), то надо применить функцию convert_to_list ниже

In [ ]:
# Если заново импортируете файл, запустите эту ячейку

def convert_to_list(image_str):
    if image_str == '[]':
        return []
    else:
        return image_str[2:-2].split("', '")

city = pd.read_csv('city.csv') # Ссылка на файл

city['useful_images'] = city['useful_images'].apply(convert_to_list)

In [22]:
def to_correct_links(links):
    global links_lst
    lst = []
    for link in links:
        if link.startswith('https:'):
            link = link
        else:
            link = 'https:'+ link
        links_lst.append(link)
        lst.append(link)
    return lst

links_lst = []
city['useful_images'].apply(to_correct_links)
print('Done')

Done


In [ ]:
# Модель та же, что и выше, поэтому, чтобы не тратить память, можно не импортировать ее
# model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)  # Перемещение модели на GPU
# processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

lst_probs = []
lst_useful_links = []

for url in tqdm(links_lst, desc="Processing images"):
    try:

        image = Image.open(requests.get(url, stream=True).raw)

        inputs = processor(
            text=["very large room", "small room or other"],
            images=image,
            return_tensors="pt",
            padding=True
        )

        inputs = {key: value.to(device) for key, value in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)

        logits_per_image = outputs.logits_per_image
        prob = round(float(logits_per_image.softmax(dim=1)[0][0]), 2)

        lst_probs.append(prob)

        if prob >= 0.8:
            lst_useful_links.append(url)

    except UnidentifiedImageError as e:
        tqdm.write(f"Could not identify image from {url}: {e}")
        continue
    except Exception as e:
        tqdm.write(f"Error processing {url}: {e}")
        continue

def to_filter_links(links):
    return [link for link in links if link in lst_useful_links]

city['large_room_images'] = city['useful_images'].apply(to_filter_links)
city['large_room_count'] = city['large_room_images'].apply(lambda x: len(x))


Processing images: 100%|██████████| 4236/4236 [05:32<00:00, 12.75it/s]


In [ ]:
# Сохранение общего файла
city.to_csv('/content/drive/MyDrive/FitnessData/Кейсы/Фото_фитнес_азия/Джакарта/jakarta_fitness_room_size.csv', index = False) # Ссылка на файл

### Применим `Image-to-Text` модель, которая делает краткие описания для изображений [nlpconnect/vit-gpt2-image-captioning](https://huggingface.co/nlpconnect/vit-gpt2-image-captioning)

Важно, чтобы в столбце useful_images был список, а не строка, поэтому если заново импортируете файл (например, при открытии в виртуальной среде), то надо применить функцию convert_to_list ниже

In [ ]:
# Если заново импортируете файл, запустите эту ячейку

def convert_to_list(image_str):
    if image_str == '[]':
        return []
    else:
        return image_str[2:-2].split("', '")

city = pd.read_csv('/content/drive/MyDrive/FitnessData/Кейсы/Фото_фитнес_азия/Джакарта/jakarta_fitness_room_size.csv') # Ссылка на файл
city['useful_images'] = city['useful_images'].apply(convert_to_list)

In [ ]:
pipe = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning", device = device)
tqdm.pandas(desc="Обработка списков изображений")

def to_text(links):
    texts = []
    for link in links:
        try:
            response = requests.get(link, stream=True)
            response.raise_for_status()
            image = Image.open(response.raw)
            texts.append(pipe(image)[0]["generated_text"])
        except UnidentifiedImageError as e:
            tqdm.write(f"Could not identify image from {url}: {e}")
            continue
        except Exception as e:
            tqdm.write(f"Error processing {url}: {e}")
            continue
    return str(texts)  # может быть пустым списком

text_lst = city["useful_images"].progress_apply(to_text).tolist()

print("Done")

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/modeling_auto.py:2242: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/982M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/982M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda


Обработка списков изображений:   0%|          | 0/1698 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (

Done


In [ ]:
def to_clear(text):
    text = str(text)
    text  =  text.replace('[', ' ')
    text  =  text.replace(']', '')
    text  =  text.replace(" '", ", ")
    text  =  text.replace(" '", "")
    text  =  text.replace("'", "")
    text  =  text.replace(",, ", "")
    return text[:-1]

city['text_descriptions'] = text_lst
city['text_descriptions'] = city['category'] + city['text_descriptions']
city['text_descriptions'] = city['text_descriptions'].apply(to_clear)

In [ ]:
# Сохранение общего файла
city.to_csv('/content/drive/MyDrive/FitnessData/Кейсы/Фото_фитнес_азия/Джакарта/jakarta_fitness_text.csv', index = False) # Ссылка на файл

### Применим `Depth Estimation` модель, которая считает глубину изображений [depth-anything/Depth-Anything-V2-Base-hf](https://huggingface.co/depth-anything/Depth-Anything-V2-Base-hf)

Важно, чтобы в столбце useful_images был список, а не строка, поэтому если заново импортируете файл (например, при открытии в виртуальной среде), то надо применить функцию convert_to_list ниже

In [ ]:
# Если заново импортируете файл, запустите эту ячейку

def convert_to_list(image_str):
    if image_str == '[]':
        return []
    else:
        return image_str[2:-2].split("', '")

city = pd.read_csv('file.csv') # ссылка на файл
city['useful_images'] = city['useful_images'].apply(convert_to_list)

In [ ]:
checkpoint = "depth-anything/Depth-Anything-V2-Base-hf"
pipe = pipeline("depth-estimation", model=checkpoint, device=device)

depth_lst = {}
total = 0

def to_depth(links):
    global total
    total += 1
    lst = []
    for link in links:
        try:
            image = Image.open(requests.get(link, stream=True).raw)
            dictt = pipe(image)
            maxx = float(dictt['predicted_depth'].max())
            ten = maxx - dictt['predicted_depth']
            lst.append(ten)
        except UnidentifiedImageError as e:
            print(f"Could not identify image from {link}: {e}")
            continue
        except Exception as e:
            print(f"Error processing {link}: {e}")
            continue
    depth_lst[total] = lst

tqdm.pandas(desc="Processing Depth Estimation")

city['useful_images'].progress_apply(to_depth)

print('DONE')

# # Сохраняем список с тензорами значений глубины
# with open('/content/drive/MyDrive/FitnessData/Кейсы/Фото_фитнес_азия/Джакарта/jakarta_depth_tensors_lst.txt', 'w') as file: # Ссылка на файл
#     for item in depth_lst:
#         file.write(f"{item}\n")

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/390M [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

Device set to use cuda

Processing Depth Estimation: 100%|██████████| 1698/1698 [07:32<00:00,  3.75it/s]

DONE


### Далее считается статистика по полученным значениям глубины. Я проводила 3 горизонтальные полоски (floor) и 3 вертикальные (wall), через все изображения, расстояние между полосками одинаковое.

Рассчитанные значения по каждой из полосок
- Максимальное значение
- Доля значений вдоль полоски не менее ее максимального значения минус 1.5 метра
- Среднеквадратическое отклонение
- Нормированная сумма

In [ ]:
depth_stats = []
for key, values in depth_lst.items():
  lst = []
  if len(values)>0:
    for ten in values:
      height = ten.shape[0]
      width = ten.shape[1]
      # maxx = round(float(ten.max()), 4)
      floor_14_max = round(float(ten[height//4].max()), 4)
      floor_12_max = round(float(ten[height//2].max()), 4)
      floor_34_max = round(float(ten[height//4*3].max()), 4)
      wall_14_max = round(float(ten[:, width//4].max()), 4)
      wall_12_max = round(float(ten[:, width//2].max()), 4)
      wall_34_max = round(float(ten[:, width//4*3].max()), 4)

      floor_14_p = round(float(len(ten[height//4][ten[height//4]>floor_14_max-1.5])/width), 4)
      floor_12_p = round(float(len(ten[height//2][ten[height//2]>floor_12_max-1.5])/width), 4)
      floor_34_p = round(float(len(ten[height//4*3][ten[height//4*3]>floor_34_max-1.5])/width), 4)
      wall_14_p = round(float(len(ten[:, width//4][ten[:, width//4]>floor_14_max-1.5])/height), 4)
      wall_12_p = round(float(len(ten[:, width//2][ten[:, width//2]>floor_12_max-1.5])/height), 4)
      wall_34_p = round(float(len(ten[:, width//4*3][ten[:, width//4*3]>floor_34_max-1.5])/height), 4)

      floor_14_std = round(float(ten[height//4].std()), 4)
      floor_12_std = round(float(ten[height//2].std()), 4)
      floor_34_std = round(float(ten[height//4*3].std()), 4)
      wall_14_std = round(float(ten[:, width//4].std()), 4)
      wall_12_std = round(float(ten[:, width//2].std()), 4)
      wall_34_std = round(float(ten[:, width//4*3].std()), 4)

      floor_14_sum_norm = round(float(ten[height//4].sum()/width), 4)
      floor_12_sum_norm = round(float(ten[height//2].sum()/width), 4)
      floor_34_sum_norm = round(float(ten[height//4*3].sum()/width), 4)
      wall_14_sum_norm = round(float(ten[:, width//4].sum()/height), 4)
      wall_12_sum_norm = round(float(ten[:, width//2].sum()/height), 4)
      wall_34_sum_norm = round(float(ten[:, width//4*3].sum()/height), 4)


      lst.append([floor_14_max, floor_12_max, floor_34_max,
                  wall_14_max, wall_12_max, wall_34_max,
                  floor_14_p, floor_12_p, floor_34_p,
                  wall_14_p, wall_12_p, wall_34_p,
                  floor_14_std, floor_12_std, floor_34_std,
                  wall_14_std, wall_12_std, wall_34_std,
                  floor_14_sum_norm, floor_12_sum_norm, floor_34_sum_norm,
                  wall_14_sum_norm , wall_12_sum_norm , wall_34_sum_norm])
  depth_stats.append(lst)

# # Сохраняем список с различными параметрами по глубине изображений
# with open('/content/drive/MyDrive/FitnessData/Кейсы/Фото_фитнес_азия/Джакарта/jakarta_depth_stats_lst.txt', 'w') as file: # Ссылка на файл
#     for item in depth_stats:
#         file.write(f"{item}\n")

In [ ]:
itog = []
for line in depth_stats:
  if len(line)>0:
    id = int(np.argmax(np.array(line)[:,0]))
    itog.append(line[id])
  else:
    itog.append([np.nan for i in range(24)])

city[['floor_14_max', 'floor_12_max', 'floor_34_max',
                  'wall_14_max', 'wall_12_max', 'wall_34_max',
                  'floor_14_p', 'floor_12_p', 'floor_34_p',
                  'wall_14_p', 'wall_12_p', 'wall_34_p',
                 'floor_14_std', 'floor_12_std', 'floor_34_std',
                  'wall_14_std', 'wall_12_std', 'wall_34_std',
                  'floor_14_sum_norm', 'floor_12_sum_norm', 'floor_34_sum_norm',
                  'wall_14_sum_norm', 'wall_12_sum_norm', 'wall_34_sum_norm']] = itog

In [ ]:
# Сохранение общего файла
city.to_csv('/content/drive/MyDrive/FitnessData/Кейсы/Фото_фитнес_азия/Джакарта/jakarta_fitness_depth_photo.csv', index = False) # Ссылка на файл

In [ ]:
city.columns

Index(['link', 'title', 'category', 'address', 'open_hours', 'popular_times',
       'website', 'phone', 'plus_code', 'review_count', 'review_rating',
       'reviews_per_rating', 'latitude', 'longitude', 'cid', 'status',
       'descriptions', 'reviews_link', 'thumbnail', 'timezone', 'price_range',
       'data_id', 'images', 'reservations', 'order_online', 'menu', 'owner',
       'complete_address', 'about', 'user_reviews', 'user_reviews_extended',
       'emails', 'images_links', 'geometry', 'area_building_osm',
       'intersect_building_osm', 'placeid', 'oldest_text', 'oldest_days',
       'category_complex', 'category_yoga', 'group_studio', 'pool', 'sauna',
       'free_parking_lot', 'paid_parking_lot', 'free_multi_storey_cp',
       'paid_multi_storey_cp', 'free_street_parking', 'paid_street_parking',
       'free_parking', 'paid_parking', 'parking', 'useful_images',
       'large_room_images', 'large_room_count', 'text_descriptions',
       'floor_14_max', 'floor_12_max', 'floo

In [ ]:
city.to_csv('/content/drive/MyDrive/FitnessData/Кейсы/Фото_фитнес_азия/Джакарта/jakarta_fitness_area_all_features.csv', index = False) # Ссылка на файл
city.to_excel('/content/drive/MyDrive/FitnessData/Кейсы/Фото_фитнес_азия/Джакарта/jakarta_fitness_area_all_features.xlsx', index = False) # Ссылка на файл